<img src="src/pighat.png" alt="drawing" width="400"/>
<h1 style='text-align:center;'><span style='color:#46aa22;'>Financial Literacy </span><span style='color:#80d162;'>AI </span><span style='color:#46aa22;'>Resource</span></h1>

---

FLAIR – Financial Literacy AI Resource – is a service that provides factual, trustworthy, easy-to-understand answers to all your financial questions. FLAIR utilizes natural language processing and state of the art AI models to provide answers to financial literacy questions. Being trained only on resources provided by the US government, the answers are fact based and bias free to support sustainable and inclusive finance. 

FLAIR compiles all the information you need into one easy to use search engine, without the pesky ads, opinion pieces, and conflicting answers that traditional search engines provide. It can be embedded in websites that are already providing resources to underserved populations, such as the formerly incarcerated, disabled, immigrant, and military communities.

Note: currently, this implementation only works on GPUs. If you're not already there, you can use a free [Google Colab](https://colab.research.google.com/github/hannahawalsh/FLAIR/blob/main/Long_Form_QA_FLAIR.ipynb) session to run this notebook.


#### _Answers are fair when you're asking FLAIR_

In [ ]:
# Clone the FLAIR repository
!git clone https://github.com/hannahawalsh/FLAIR.git

In [ ]:
%%capture
# Install dependencies - this takes a few moments
!pip install transformers
!pip install faiss_gpu
!pip install datasets

In [ ]:
# Imports 
import sys
sys.path.insert(0, "./FLAIR")
from LFQA_utils import longFormQA, wrap_print

### The Long Form Question Answering Model
A long form question answering model is an AI model that provides long, complex answers to questions. Unlike other question answering models, it does not simply pull snippets of text from a database for its answers, but synthesizes information to create one cohesive answer. It can answer more abstract questions rather than trivia-type ones.  

Our model actually consists of two parts. One is the question-answer model, which pulls passages of text relating to the answer. The second is a sequence to sequence model, which forms those answers into one cohesive answer. 

Some of the functions used in this project are modified from the blogpost and accompanying github code [Explain _Anything_ Like I'm Five: A Model for Open Domain Long Form Question Answering](https://yjernite.github.io/lfqa.html), which goes more in depth on the technical aspects of this type of model. We applied their concepts to fit the needs of our project and apply it to the financial literacy world.  

---

#### Loading the model and the data
Run the below cell to load the model weights from Hugging Face 🤗 and index the financial literacy dataset (if that file doesn't already exist). It might take a little bit if the data hasn't been indexed. You only need to run this cell once per session.

In [ ]:
# Note: this cell takes ~2 minutes to run
if "lfqa" not in locals():
    # Initialize our Long-Form Question-Answering class
    data_filename = "https://github.com/hannahawalsh/FLAIR/raw/main/financial_literacy_data.csv"
    lfqa = longFormQA(data_filename, encode_length=128)

    # Load model weights from hugging face
    qa_model_name = "yjernite/retribert-base-uncased"
    s2s_model_name = "yjernite/bart_eli5"
    lfqa.load_model_weights(qa_model_name, s2s_model_name)

    # Create a dense index of our data if one hasn't been created yet
    lfqa.create_dense_index(batch_size=64)

### And we're ready to roll!
You can now ask the model questions! Try it out below.

Change how the answers are formed by changing some of the custom parameters.

Do note that because of the limited time and resources of the hackathon, the answers may not always make that much sense or be accurate. But it's really good at the topic for which it has a lot of data 🙂

In [ ]:
#@title Ask it a question  { run: "auto" }
question = "What is a credit score?" #@param {type:"string"}
answer = lfqa.ask_a_question(question)
wrap_print(answer)

In [ ]:
#@title Ask a question with custom parameters  { run: "auto" }
question = "Should I have a savings account?" #@param {type:"string"}
max_question_len = 1024 #@param {type:"integer"}
min_answer_len = 256 #@param {type:"integer"}
max_answer_len = 512 #@param {type:"integer"}

answer = lfqa.ask_a_question(
    question,
    max_question_len=max_question_len,
    min_answer_len=min_answer_len,
    max_answer_len=max_answer_len
)
wrap_print(answer)

---
Below are a few more examples. Feel free to change the cells to experiment yourself. Note the model is lacking information in some areas so it doesn't answer well, or even correctly. Also watch for how the model parameters affect the answers.

In [ ]:
# Here's a few more examples
kwargs = {
    "max_question_len": 1024, 
    "min_answer_len": 64, 
    "max_answer_len": 512} 

questions = [
    "Are there budgeting tools?",
    "What if I run out of money?",
    "What if I don't have a credit score?",
    "What's the difference between a savings account and a checking account?",
    "What if I run out of money?"
]

lfqa.ask_questions(questions, kwargs)

In [ ]:
# Ask those same questions but with different model parameters
kwargs = {
    "max_question_len": 256, 
    "min_answer_len": 16, 
    "max_answer_len": 256} 
lfqa.ask_questions(questions, kwargs)

In [ ]:
# Try some more questions
questions = [
    "How do I open up a bank account?",
    "What do I need a bank account for?",
    "Why should I open a savings account?",
    "Whats the difference between a savings and checking account?",
    "Whats the difference between credit and debit?",
]
kwargs = {"max_question_len": 256, "min_answer_len": 32, "max_answer_len": 512} 
lfqa.ask_questions(questions, kwargs)

In [ ]:
# And those same ones with different parameters
kwargs = {"max_question_len": 256, "min_answer_len": 32, "max_answer_len": 512} 
lfqa.ask_questions(questions, kwargs)

In [ ]:
# These answers are using different parameters and it is clear that we 
# need to increase min_answer_len
# When answers are too short, they aren't helpful
kwargs = {"max_question_len": 1024, "min_answer_len": 8, "max_answer_len": 1024} 
questions = [
    "What do I do if I'm poor?",
    "How do I get out of debt?",
    "Can I get a credit card without a credit score?",
    "How do I improve my credit score?",
    "Are there bank accounts for non-citizens?"]
lfqa.ask_questions(questions, kwargs)